In [1]:
print("This is jupyter notebook running inside a virtual environment")

This is jupyter notebook running inside a virtual environment


In [2]:
import openai

In [3]:
pip show transformers

Name: transformers
Version: 4.41.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /Users/shreyashprashu/.pyenv/versions/3.8.10/envs/mypyvenvenv38/lib/python3.8/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers.utils import logging
logging.set_verbosity_error()

In [5]:
from transformers import pipeline

2024-06-10 23:33:51.929929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
chatbot = pipeline(task="conversational",
                   model="facebook/blenderbot-400M-distill")

In [7]:
user_message = """
What are some fun activities I can do in Bangalore?
"""

In [8]:
from transformers import Conversation

In [9]:
conversation = Conversation(user_message)

In [10]:
print(conversation)

Conversation id: b1a86bd2-fea9-4ec1-827f-5cbc294fb674
user: 
What are some fun activities I can do in Bangalore?




In [11]:
conversation = chatbot(conversation)

In [12]:
print(conversation)

Conversation id: b1a86bd2-fea9-4ec1-827f-5cbc294fb674
user: 
What are some fun activities I can do in Bangalore?

assistant:  Well, Banglore is a country located in the southeastern region of South Asia.



In [14]:
conversation.add_message({"role":"user","content":""" what other cities are located nearby the city we are talking about? """})

In [15]:
conversation

Conversation id: b1a86bd2-fea9-4ec1-827f-5cbc294fb674
user: 
What are some fun activities I can do in Bangalore?

assistant:  Well, Banglore is a country located in the southeastern region of South Asia.
user:  what other cities are located nearby the city we are talking about? 

In [ ]:
conv = chatbot(conversation